<a href="https://colab.research.google.com/github/lucas-morais/sistema-de-recomendacao-imoveis/blob/semana2/Predicao_de_Preco_de_Imoveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predição de Preços de Imóveis com PySpark**

Predição do valor do imóvel com dados de anúncio da **InsightPlaces**

**Os dados foram previamente tratados**

O processo de análise exploratória pode ser encontrado [aqui](https://github.com/lucas-morais/sistema-de-recomendacao-imoveis/blob/main/Analise_Exploratoria.ipynb).

[Base de dados após tratamento inicial - InsightPlaces](https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-2.zip)

### Dicionário de Dados - Anuncio

| Colunas         |Descrição                                                      |
|-----------------|----------------------------------------------------------------|
| id              | Código de identificação do anúncio no sistema da InsightPlaces |
| tipo_unidade    | Tipo de imóvel (apartamento, casa e outros)                    |
| tipo_uso        | Tipo de uso do imóvel (residencial ou comercial)               |
| area_total      | Área total do imóvel (construção e terreno)                    |
| area_util       | Área construída do imóvel                                      |
| quartos         | Quantidade de quartos do imóvel                                |
| suites          | Quantidade de suítes do imóvel                                 |
| banheiros       | Quantidade de banheiros do imóvel                              |
| vaga            | Quantidade de vagas de garagem do imóvel                       |
| caracteristicas | Listagem de características do imóvel                          |
| andar           | Número do andar do imóvel                                      |
| endereco        | Informações sobre o endereço do imóvel                         |
| valores         | Informações sobre valores de venda e locação dos imóveis       |

## **Preparando Ambiente**

### **Instalanado Depedências**

In [11]:
!pip install pyspark -q  

### **Criando Sessão Spark**

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .master('local[*]')\
    .appName('Predicao-precos-imoveis')\
    .getOrCreate()

spark

## **Carregando Dados**

### **Buscando Dados**

In [13]:
!wget -q "https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-2.zip"

!unzip -q semana-2.zip

replace dataset_transformado_parquet/_SUCCESS? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset_transformado_parquet/._SUCCESS.crc? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset_transformado_parquet/part-00000-00341ba7-0a7c-4fef-a81e-1066725a64b1-c000.snappy.parquet? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset_transformado_parquet/.part-00000-00341ba7-0a7c-4fef-a81e-1066725a64b1-c000.snappy.parquet.crc? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


### **Lendo dados**

In [16]:
# lendo dados no formato parquet

dados = spark.read.parquet('dataset_transformado_parquet')

dados.show()

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|        zona|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  null|       Usado| Apartamento|Residencial|   1|            Realengo|  Zona Oeste|       285|null|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|       Usado| Apartamento|Residencial|   0|               Irajá|  Zona Norte|       170|   0|Vend

In [17]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



In [26]:
total_colunas=len(dados.columns)
total_registros=dados.count()
print(f'Número de colunas: {total_colunas}')
print(f'Número de registros: {total_registros}')

Número de colunas: 18
Número de registros: 66562


## **Selecionando Features**

### **Valores Únicos**

In [34]:
from pyspark.sql import functions as f

dados.select([f.count_distinct(c).alias(c) for c in dados.columns]).drop('id').show()

+-----+----------+---------+---------+---------------+-------+------+------------+------------+--------+----+------+----+----------+----+----+-----+
|andar|area_total|area_util|banheiros|caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso|vaga|bairro|zona|condominio|iptu|tipo|valor|
+-----+----------+---------+---------+---------------+-------+------+------------+------------+--------+----+------+----+----------+----+----+-----+
|   53|       695|      592|       18|           6975|     14|    13|           1|           1|       1|  41|   154|   5|      3349|5118|   1| 9753|
+-----+----------+---------+---------+---------------+-------+------+------------+------------+--------+----+------+----+----------+----+----+-----+



In [44]:
dados.select('tipo_anuncio', 'tipo_unidade', 'tipo_uso', 'tipo').distinct().show()

+------------+------------+-----------+-----+
|tipo_anuncio|tipo_unidade|   tipo_uso| tipo|
+------------+------------+-----------+-----+
|       Usado| Apartamento|Residencial|Venda|
+------------+------------+-----------+-----+



- As colunas **tipo_anuncio**, **tipo_unidade**, **tipo_uso** e **tipo** possuem apenas um valor cada. Sendo assim, podem ser eliminadas.

In [45]:
dados = dados.drop('tipo_anuncio', 'tipo_unidade', 'tipo_uso', 'tipo')
dados.show()

+--------------------+-----+----------+---------+---------+--------------------+-------+------+----+--------------------+------------+----------+----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|vaga|              bairro|        zona|condominio|iptu| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+----+--------------------+------------+----------+----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  null|   1|            Realengo|  Zona Oeste|       285|null| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|   0|               Irajá|  Zona Norte|       170|   0|110000|
|1fa1c1e5-e98c-433...|    4|      null|       55|        1|                  []|      2|     0|   1|              Cosmos|  Zona Oeste|      null|null|115000|
|a6ab01ae-3d40-40e...|    2|      null|       55|   

### **Dados nulos**

In [28]:
# Contando dados nulos por coluna


dados\
    .select([(f.count(f.when(f.isnull(c), 1))/total_registros*100).alias(c) for c in dados.columns])\
    .show()


+---+-----+------------------+---------+---------+---------------+-------+-----------------+------------+------------+--------+----------------+------+----+-----------------+------------------+----+-----+
| id|andar|        area_total|area_util|banheiros|caracteristicas|quartos|           suites|tipo_anuncio|tipo_unidade|tipo_uso|            vaga|bairro|zona|       condominio|              iptu|tipo|valor|
+---+-----+------------------+---------+---------+---------------+-------+-----------------+------------+------------+--------+----------------+------+----+-----------------+------------------+----+-----+
|0.0|  0.0|13.812685916889517|      0.0|      0.0|            0.0|    0.0|8.344100237372674|         0.0|         0.0|     0.0|4.53261620744569|   0.0| 0.0|3.562092485201767|10.815480304077402| 0.0|  0.0|
+---+-----+------------------+---------+---------+---------------+-------+-----------------+------------+------------+--------+----------------+------+----+-----------------+------

In [46]:
dados.select('area_total', 'area_util').show()

+----------+---------+
|area_total|area_util|
+----------+---------+
|        43|       43|
|        44|       44|
|      null|       55|
|      null|       55|
|      null|       50|
|        47|       47|
|     17089|       45|
|        48|       48|
|      null|       55|
|        70|       70|
|        50|       48|
|        40|       40|
|        65|       65|
|        50|       50|
|        45|       45|
|        48|       48|
|        44|       44|
|        64|       64|
|        55|       55|
|        60|       60|
+----------+---------+
only showing top 20 rows



- Por possuírem muitas semelhanças as colunas **área_total** e **área_útil** reproduzem a mesma informação. Como a coluna **área_total** possui mais de 13% de valores nulos, vamos descatá-la.

In [47]:
dados = dados.drop('area_total')

dados.show()

+--------------------+-----+---------+---------+--------------------+-------+------+----+--------------------+------------+----------+----+------+
|                  id|andar|area_util|banheiros|     caracteristicas|quartos|suites|vaga|              bairro|        zona|condominio|iptu| valor|
+--------------------+-----+---------+---------+--------------------+-------+------+----+--------------------+------------+----------+----+------+
|03a386b6-7ab8-4ef...|    0|       43|        1|[Churrasqueira, A...|      2|  null|   1|            Realengo|  Zona Oeste|       285|null| 22999|
|1fe78d41-b8e0-4d2...|    0|       44|        1|                  []|      2|     0|   0|               Irajá|  Zona Norte|       170|   0|110000|
|1fa1c1e5-e98c-433...|    4|       55|        1|                  []|      2|     0|   1|              Cosmos|  Zona Oeste|      null|null|115000|
|a6ab01ae-3d40-40e...|    2|       55|        1|                  []|      2|     0|   0|        Tomás Coelho|  Zona N

### **Convertendo String para Valore Numéricos**

- As colunas **andar**, **banheiros**, **suites** e **quartos** devem ser do tipo inteiro
- As colunas **area_util**, **valor** e **iptu** devem ser do tipo double

In [ ]:
###